 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Prepare-the-text" data-toc-modified-id="Prepare-the-text-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare the text</a></span></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Prepare-the-input-for-the-model" data-toc-modified-id="Prepare-the-input-for-the-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare the input for the model</a></span></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Set-up-the-model" data-toc-modified-id="Set-up-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up the model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Stateless-model" data-toc-modified-id="Stateless-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Stateless model</a></span></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Stateful-model-with-regular-input" data-toc-modified-id="Stateful-model-with-regular-input-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Stateful model with regular input</a></span></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Stateful-model-with-restructured-input" data-toc-modified-id="Stateful-model-with-restructured-input-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Stateful model with restructured input</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Copy-weights-to-prediction-model" data-toc-modified-id="Copy-weights-to-prediction-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Copy weights to prediction model</a></span></li><li><span><a href="http://localhost:8888/notebooks/hp_generator.ipynb#Test-Model" data-toc-modified-id="Test-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Test Model</a></span></li></ul></div>

In [2]:
# Import 
import numpy as np
from numpy.random import choice
import keras
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, \
    TimeDistributed, Embedding, Input
from keras.models import Model, Sequential
from keras.optimizers import Adam
from __future__ import division, print_function

Using TensorFlow backend.


## Prepare the text

- `char` are all the unique characters.
- `vocab_size` is the number of unique characters
- `idx` is the text as a list
- `char2idx` and `idx2char` are the conversion dictionaries

In [3]:
path = '/Users/stephanrasp/repositories/courses/data/hp/'
fn = 'HP_7_-_Harry_Potter_and_the_Deathly_Hallows.txt'

In [4]:
text = open(path + fn).read()

In [38]:
# Create smaller sample
text = text[:200000]

In [39]:
len(text)

200000

In [63]:
print(text[10000:10200])

ally he is certain,’ said Snape. ‘I assure you, Yaxley, the Auror Office will play no further part in the protection of Harry Potter. The Order believes that we have infiltrated the Ministry.’



In [41]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars), len(chars)

['\t', '\n', '\r', ' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x93', '\x94', '\x98', '\x99', '\x9c', '\x9d', '\xa4', '\xa6', '\xa9', '\xc2', '\xc3', '\xe2']


(None, 89)

In [42]:
# Build dictionaries
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

In [43]:
print([char2idx[c] for c in text[10000:10050]])

[50, 61, 61, 74, 3, 57, 54, 3, 58, 68, 3, 52, 54, 67, 69, 50, 58, 63, 8, 88, 76, 80, 3, 68, 50, 58, 53, 3, 42, 63, 50, 65, 54, 10, 3, 88, 76, 79, 32, 3, 50, 68, 68, 70, 67, 54, 3, 74, 64, 70]


In [44]:
# Convert the entire text
idx = [char2idx[c] for c in text]

In [45]:
len(idx)

200000

## Prepare the input for the model

As input we need 

In [106]:
# cs = 40
cs = 15
bs = 64

In [74]:
len(idx)

200000

In [108]:
idx_cropped = idx[:(len(idx) // cs * cs) + 1]

In [110]:
xs = np.reshape(idx_cropped[:-1], (-1, cs))
ys = np.reshape(idx_cropped[1:], (-1, cs))
xs.shape, ys.shape

((13333, 15), (13333, 15))

In [112]:
xs[:4]

array([[31, 24, 41, 41, 48,  2,  1,  2,  1,  2,  1, 39, 38, 43, 43],
       [28, 41,  2,  1,  2,  1,  2,  1, 50, 63, 53,  3, 69, 57, 54],
       [ 3, 27, 54, 50, 69, 57, 61, 74,  3, 31, 50, 61, 61, 64, 72],
       [68,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1, 33, 10]])

In [113]:
ys[:4]

array([[24, 41, 41, 48,  2,  1,  2,  1,  2,  1, 39, 38, 43, 43, 28],
       [41,  2,  1,  2,  1,  2,  1, 50, 63, 53,  3, 69, 57, 54,  3],
       [27, 54, 50, 69, 57, 61, 74,  3, 31, 50, 61, 61, 64, 72, 68],
       [ 2,  1,  2,  1,  2,  1,  2,  1,  2,  1,  2,  1, 33, 10, 34]])

In [114]:
# Crop to batch size
xs = xs[:(xs.shape[0] // bs * bs)]
ys = ys[:(ys.shape[0] // bs * bs)]
xs.shape, ys.shape

((13312, 15), (13312, 15))

In [117]:
xs_state = np.zeros(xs.shape)
ys_state = np.zeros(ys.shape)

In [118]:
n_batch = xs.shape[0] // bs
n_batch

208

In [119]:
for i in range(bs):
    xs_state[i::bs] = xs[i*n_batch:(i+1)*n_batch]
    ys_state[i::bs] = ys[i*n_batch:(i+1)*n_batch]

In [120]:
ys.shape, ys_state.shape

((13312, 15), (13312, 15))

In [121]:
ys = np.atleast_3d(ys)
ys_state = np.atleast_3d(ys_state)
ys.shape, ys_state.shape

((13312, 15, 1), (13312, 15, 1))

So Ys is just shifted by one!

## Set up the model

### Stateless model

In [56]:
def build_model(vocab_size, n_fac, cs, batch_size, n_hidden, stateful):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=n_fac, input_length=cs, 
                  batch_input_shape=(batch_size, cs)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=stateful, dropout_U=0.2,
             dropout_W=0.2),
        LSTM(n_hidden, return_sequences=True, stateful=stateful, dropout_U=0.2,
             dropout_W=0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.1),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
    model.compile(Adam(), loss='sparse_categorical_crossentropy')
    return model

In [57]:
n_fac, n_hidden = (12, 124)

In [58]:
model = build_model(vocab_size, n_fac, cs, bs, n_hidden, stateful=False)

In [59]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (64, 15, 12)          1068        embedding_input_2[0][0]          
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (64, 15, 12)          48          embedding_2[0][0]                
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (64, 15, 124)         67952       batchnormalization_2[0][0]       
____________________________________________________________________________________________________
lstm_4 (LSTM)                    (64, 15, 124)         123504      lstm_3[0][0]                     
___________________________________________________________________________________________

In [61]:
model.fit(xs, ys, batch_size=bs, nb_epoch=11, shuffle=False)

Epoch 1/11
13312/13312 [==============================] - 43s - loss: 2.3580    
Epoch 2/11
13312/13312 [==============================] - 41s - loss: 2.2104    
Epoch 3/11
13312/13312 [==============================] - 41s - loss: 2.1218    
Epoch 4/11
13312/13312 [==============================] - 46s - loss: 2.0544    
Epoch 5/11
13312/13312 [==============================] - 42s - loss: 2.0039    
Epoch 6/11
13312/13312 [==============================] - 42s - loss: 1.9600    
Epoch 7/11
13312/13312 [==============================] - 43s - loss: 1.9226    
Epoch 8/11
13312/13312 [==============================] - 41s - loss: 1.8915    
Epoch 9/11
13312/13312 [==============================] - 41s - loss: 1.8602      ETA - ETA: 6s - l - E
Epoch 10/11
13312/13312 [==============================] - 42s - loss: 1.8359    
Epoch 11/11
13312/13312 [==============================] - 41s - loss: 1.8147    


### Stateful model with regular input

In [127]:
def train_stateful(epochs, model, xs, ys):
    for e in xrange(epochs):
        model.reset_states()
        h = model.fit(xs, ys, batch_size=bs, nb_epoch=1, shuffle=False)
        print(h.history['loss'])

In [123]:
model_state = build_model(vocab_size, n_fac, cs, bs, n_hidden, stateful=True)

In [126]:
train_stateful(12, model_state, xs, ys)

Epoch 1/1
13312/13312 [==============================] - 44s - loss: 2.9338    
[2.9337928020037136]
Epoch 1/1
13312/13312 [==============================] - 44s - loss: 2.3585    - ETA: 3
[2.3584547570118537]
Epoch 1/1
13312/13312 [==============================] - 42s - loss: 2.2393      
[2.2392862335993695]
Epoch 1/1
13312/13312 [==============================] - 47s - loss: 2.1666    
[2.1665542996846714]
Epoch 1/1
 5824/13312 [============>.................] - ETA: 24s - loss: 2.1878

KeyboardInterrupt: 

### Stateful model with restructured input

In [128]:
model_state2 = build_model(vocab_size, n_fac, cs, bs, n_hidden, stateful=True)

In [129]:
train_stateful(4, model_state2, xs_state, ys_state)

Epoch 1/1
13312/13312 [==============================] - 43s - loss: 2.8923    
[2.8922764922563848]
Epoch 1/1
13312/13312 [==============================] - 43s - loss: 2.2540    
[2.2539829886876621]
Epoch 1/1
13312/13312 [==============================] - 43s - loss: 2.1044    
[2.1043892422547708]
Epoch 1/1
13312/13312 [==============================] - 44s - loss: 2.0107    
[2.0107039539859843]


## Copy weights to prediction model

In [ ]:
def copy_weights(model):
    weights = model.get_weights()
    model_pred = build_model(vocab_size, n_fac, cs, 1, n_hidden, stateful=False)
    model_pred.set_weights(weights)
    return model_pred

## Test Model

In [72]:
def print_example(seed_string, len_seq):
    for i in range(len_seq):
        x=np.array([char2idx[c] for c in seed_string[-cs:]])[np.newaxis,:]
        preds = model_pred.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [65]:
seed = 'Harry picked up Hedwig’s cage, his Firebolt and his rucksack, gave his unnaturally tidy bedroom one last sweeping look and then made his ungainly way back downstairs to the hall, where he deposited cage, broomstick and bag near the foot of the stairs. The light'
print(seed)

Harry picked up Hedwig’s cage, his Firebolt and his rucksack, gave his unnaturally tidy bedroom one last sweeping look and then made his ungainly way back downstairs to the hall, where he deposited cage, broomstick and bag near the foot of the stairs. The light


In [66]:
seed[-40:]

'g near the foot of the stairs. The light'

In [73]:
print_example(seed, 320)

Harry picked up Hedwig’s cage, his Firebolt and his rucksack, gave his unnaturally tidy bedroom one last sweeping look and then made his ungainly way back downstairs to the hall, where he deposited cage, broomstick and bag near the foot of the stairs. The light ufownnave other wize glear somether tto it was deepiour sup, in a mills shem how were be a said Durnon was said a son be they dall befort coppelse And Harry. ‘Not have looking and as thome cace.

How, blanfur Datter applaise Garly.

‘WeR’s gessack.

‘That Holdemtort of they was Pomise un – we was licks
